# Solving a Sudoku puzzle with z3


In [62]:
!pip install z3-solver
from z3 import *

Create an Integer Variable for each cell of the Sudoku grid:

In [63]:
X = [[Int(f"x_{i + 1}_{j + 1}") for j in range(9)] for i in range(9)]

In [64]:
X

[[x_1_1, x_1_2, x_1_3, x_1_4, x_1_5, x_1_6, x_1_7, x_1_8, x_1_9],
 [x_2_1, x_2_2, x_2_3, x_2_4, x_2_5, x_2_6, x_2_7, x_2_8, x_2_9],
 [x_3_1, x_3_2, x_3_3, x_3_4, x_3_5, x_3_6, x_3_7, x_3_8, x_3_9],
 [x_4_1, x_4_2, x_4_3, x_4_4, x_4_5, x_4_6, x_4_7, x_4_8, x_4_9],
 [x_5_1, x_5_2, x_5_3, x_5_4, x_5_5, x_5_6, x_5_7, x_5_8, x_5_9],
 [x_6_1, x_6_2, x_6_3, x_6_4, x_6_5, x_6_6, x_6_7, x_6_8, x_6_9],
 [x_7_1, x_7_2, x_7_3, x_7_4, x_7_5, x_7_6, x_7_7, x_7_8, x_7_9],
 [x_8_1, x_8_2, x_8_3, x_8_4, x_8_5, x_8_6, x_8_7, x_8_8, x_8_9],
 [x_9_1, x_9_2, x_9_3, x_9_4, x_9_5, x_9_6, x_9_7, x_9_8, x_9_9]]

Requirement 1: each cell must contain a digit (1 to 9)

In [65]:
cells_c  = [And(X[i][j] > 0, X[i][j] < 10) for i in range(9) for j in range(9) ]

Requirement 2: Every digit has to be placed exactly once in each row:

In [67]:
rows_c   = [ Distinct([ X[i][j0] 
                        for i in range(9)]) 
            for j0 in range(9) ]

Requirement 3: Every digit has to be placed exactly once in each row:

In [68]:
cols_c   = [ Distinct([ X[i0][j] 
                        for j in range(9)]) 
             for i0 in range(9)]

Requirement 4: Every digit has to be placed exactly once in each $3 \times 3$ subgrid

In [69]:
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j] 
                        for i in range(3) for j in range(3) ]) 
             for i0 in range(3) for j0 in range(3) ]

Now, put all the requirements together

In [70]:
sudoku_c = cells_c + rows_c + cols_c + sq_c

and consider some concrete puzzle:

In [71]:
instance = ((5,3,0,0,7,0,0,0,0),
            (6,0,0,1,9,5,0,0,0),
            (0,9,8,0,0,0,0,6,0),
            (8,0,0,0,6,0,0,0,3),
            (4,0,0,8,0,3,0,0,1),
            (7,0,0,0,2,0,0,0,6),
            (0,6,0,0,0,0,2,8,0),
            (0,0,0,4,1,9,0,0,5),
            (0,0,0,0,8,0,0,7,9))

and push the filled spaces and a requirement:

In [72]:
instance_c = [ If(instance[i][j] == 0, 
                  True, 
                  X[i][j] == instance[i][j]) 
               for i in range(9) for j in range(9) ]

Build a z3 specification: 

In [73]:
s = Solver()                                            # (1)
s.add(sudoku_c + instance_c)                            # (2)


Check its satisfability:

In [74]:
s.check()

sat

Print the solution (if sat):

In [75]:
if s.check() == sat:                                    # (3)
    m = s.model()                                       # (4)
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]     # (5)
          for i in range(9) ]
    print_matrix(r)                                     # (6)
else:
    print("failed to solve")        

[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]


# Exercise

Prepare your z3 model to solve generic sudokus, i.e. puzzles with sizes $k^2 \times k^2$, where $k$ is the number of different digits involved.



Then, apply it to solve the following puzzles of sizes $2^2 \times 2^2$, $3^2 \times 3^2$ and $4^2 \times 4^2$


In [76]:
instance2 = ((1,0,0,0),
             (0,2,0,0),
             (0,0,3,0),
             (0,0,0,4))

In [77]:
instance3 = ((5,3,0,0,7,0,0,0,0),
            (6,0,0,1,9,5,0,0,0),
            (0,9,8,0,0,0,0,6,0),
            (8,0,0,0,6,0,0,0,3),
            (4,0,0,8,0,3,0,0,1),
            (7,0,0,0,2,0,0,0,6),
            (0,6,0,0,0,0,2,8,0),
            (0,0,0,4,1,9,0,0,5),
            (0,0,0,0,8,0,0,7,9))

In [78]:
instance4=((15,0,1,0,0,3,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0),
           (0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0))